In [60]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import json
import os
import pickle
import EarlyExitModel
import torch.nn as nn

In [3]:
# Check if PyTorch is using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch is using: {device}")

# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Test a simple PyTorch tensor operation on the GPU
x = torch.tensor([2.0, 3.0], device=device)
y = torch.tensor([4.0, 5.0], device=device)
result = x * y
print(f"Result of tensor operation: {result}")

PyTorch is using: cuda
PyTorch version: 2.2.1+cu121
Result of tensor operation: tensor([ 8., 15.], device='cuda:0')


In [206]:
LAMDBA_ = 0.1

In [4]:
orgmodel = models.resnet50(pretrained=True)
last_layer = orgmodel.fc

c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
orgmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
last_layer

Linear(in_features=2048, out_features=1000, bias=True)

In [6]:
with open("imagenet_class_index.json", "r") as jf:
    class_index_json = json.load(jf)
print(class_index_json)
class_indices = {}
for k, v in class_index_json.items():
    class_indices[v[0]] = int(k)
print(class_indices)

{'0': ['n01440764', 'tench'], '1': ['n01443537', 'goldfish'], '2': ['n01484850', 'great_white_shark'], '3': ['n01491361', 'tiger_shark'], '4': ['n01494475', 'hammerhead'], '5': ['n01496331', 'electric_ray'], '6': ['n01498041', 'stingray'], '7': ['n01514668', 'cock'], '8': ['n01514859', 'hen'], '9': ['n01518878', 'ostrich'], '10': ['n01530575', 'brambling'], '11': ['n01531178', 'goldfinch'], '12': ['n01532829', 'house_finch'], '13': ['n01534433', 'junco'], '14': ['n01537544', 'indigo_bunting'], '15': ['n01558993', 'robin'], '16': ['n01560419', 'bulbul'], '17': ['n01580077', 'jay'], '18': ['n01582220', 'magpie'], '19': ['n01592084', 'chickadee'], '20': ['n01601694', 'water_ouzel'], '21': ['n01608432', 'kite'], '22': ['n01614925', 'bald_eagle'], '23': ['n01616318', 'vulture'], '24': ['n01622779', 'great_grey_owl'], '25': ['n01629819', 'European_fire_salamander'], '26': ['n01630670', 'common_newt'], '27': ['n01631663', 'eft'], '28': ['n01632458', 'spotted_salamander'], '29': ['n01632777', 

In [7]:
imagenette_classes = [0, 217, 482, 491, 497, 566, 569, 571, 574, 701]
update_class_indices = {}
for i in range(len(imagenette_classes)):
    update_class_indices[imagenette_classes[i]] = i
print(update_class_indices)

{0: 0, 217: 1, 482: 2, 491: 3, 497: 4, 566: 5, 569: 6, 571: 7, 574: 8, 701: 9}


In [9]:
with open("..\\ee_models\\resnet_alpha075.pkl", "rb") as f:
    model075 = pickle.load(f)
model075.eval()
model075.to(device)

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): OptionalExitModule(
      (module): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [204]:
def get_ee_classification(classname, filename, model, targetfol):
    image = Image.open(f"..\\torch_normal_adv_samples\\{targetfol}\\outputs\\{classname}\\{filename}")
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        predclass = torch.argmax(output, dim=1).item()
    # print(model075.num_exits_per_module)
    return predclass

In [211]:
def get_ee_classification_set(directory_path, model, targetfol):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        expected = update_class_indices[class_indices[classname]]
        cnt = 0
        correct = 0
        # print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)][:100]
        for file in files:
            filename = file.split("\\")[-1]
            eec = get_ee_classification(classname, filename, model, targetfol)
            if expected == eec:
                correct += 1
            cnt += 1
        print(f"Class: {classname} has got accuracy of {correct / cnt * 100}%, {correct, cnt}")
            

In [212]:
# with alpha = 0.75 model
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer\\outputs", model075, "results_transfer")

tensor([[-61.8233, -87.9044, -85.0726, -89.6248, -86.0755, -87.5564, -84.3862,
         -86.4924, -86.0239, -89.8337]], device='cuda:0')
tensor([[-51.7873, -81.6988, -81.0128, -80.6667, -76.3256, -80.5878, -84.9997,
         -84.5765, -86.2887, -82.4307]], device='cuda:0')
tensor([[ -74.6242, -103.3291, -105.1690, -110.3349, -104.5696, -108.1994,
         -105.4602, -100.7808, -107.8344, -100.1635]], device='cuda:0')
tensor([[-60.7224, -89.0302, -88.7166, -90.9149, -94.0321, -94.8080, -92.6116,
         -86.3290, -90.3520, -90.7051]], device='cuda:0')
tensor([[ -65.4293,  -96.3524,  -94.5010, -101.0553,  -98.3590,  -94.4632,
          -96.3694,  -95.6115,  -98.0271,  -97.2993]], device='cuda:0')
tensor([[ -84.5254, -116.7646, -117.2840, -109.2164, -118.9134, -118.9899,
         -119.9564, -115.1784, -113.0575, -123.3325]], device='cuda:0')
tensor([[ -68.9897,  -96.9935,  -94.7755,  -87.0891, -101.0238, -101.8186,
          -95.9195,  -98.2049,  -98.3245, -100.8295]], device='cuda:0')
C

In [205]:
# with alpha = 0.75 model
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer_05\\outputs", model075, "results_transfer_05")

tensor([[ -93.6339, -118.0303, -121.4124, -115.4905, -117.2157, -123.6992,
         -134.0484, -120.6213, -127.5439, -121.0155]], device='cuda:0')
tensor([[-118.3969, -132.9406, -134.5268, -121.1990, -106.6135, -131.8404,
         -149.5243, -128.2952, -133.9277, -127.9541]], device='cuda:0')
tensor([[ -74.9183, -103.6233,  -97.2099, -102.7347, -101.1025, -105.9462,
         -111.3982, -104.5521, -101.1545, -109.6511]], device='cuda:0')
tensor([[-106.3666, -131.1542, -132.6691, -106.1938, -128.7982, -134.4903,
         -141.8931, -129.8551, -130.7430, -140.8376]], device='cuda:0')
Class: n01440764 has got accuracy of 25.0%
tensor([[-140.7077, -117.4939, -140.3585, -123.9676, -129.2942, -136.2652,
         -145.7380, -140.8042, -144.8155, -141.0303]], device='cuda:0')
tensor([[ -97.0467,  -73.4644, -101.3968, -101.0045,  -91.7618,  -98.0257,
         -108.7271, -100.1662,  -99.1860, -105.8787]], device='cuda:0')
tensor([[-101.7150,  -70.3499, -106.7268, -110.9430, -100.9663, -107.5292,


In [27]:
with open("..\\ee_models\\resnet_alpha025.pkl", "rb") as f:
    model025 = pickle.load(f)
model025.eval()
model025.to(device)

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): OptionalExitModule(
      (module): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [213]:
# with alpha = 0.25 model
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer\\outputs", model025, "results_transfer")

tensor([[ -58.9473,  -94.6308,  -92.4328,  -99.0249,  -87.6698,  -95.4554,
         -101.6631, -100.2805, -100.6912,  -95.1772]], device='cuda:0')
tensor([[ -73.6214, -101.8108,  -96.7806, -100.0202, -101.9280,  -99.6308,
          -96.5056, -104.1047, -106.6515, -103.9540]], device='cuda:0')
tensor([[ -93.5988, -124.3688, -123.4507, -119.7155, -127.8525, -128.8710,
         -133.4921, -124.4919, -123.8182, -126.4445]], device='cuda:0')
tensor([[ -74.6386, -105.8584, -108.3181, -103.1112, -107.2387, -105.6945,
         -110.0735, -105.4213, -107.5408, -114.6649]], device='cuda:0')
tensor([[-54.8234, -87.7530, -84.6365, -94.1088, -84.4600, -90.1654, -88.9537,
         -91.7660, -91.0761, -91.0705]], device='cuda:0')
tensor([[ -64.8027,  -94.8109,  -93.7087,  -94.8529,  -99.3714,  -95.5270,
          -93.5688,  -89.0274,  -95.7808, -101.1163]], device='cuda:0')
tensor([[ -64.7672,  -98.5732, -100.1071,  -95.1747, -102.2758,  -98.0056,
          -97.6950,  -97.4688,  -98.9899, -104.3329]]

In [199]:
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer_05\\outputs", model025, "results_transfer_05")

tensor([[-111.0163, -126.3423, -127.3803, -132.9470,  -99.7072, -129.2703,
         -144.4127, -136.2668, -130.7248, -120.3402]], device='cuda:0')
tensor([[-101.3505, -126.3998, -121.9231, -111.3339, -102.7905, -112.8490,
         -129.9386, -125.4684, -133.5619, -129.8259]], device='cuda:0')
tensor([[-129.1718, -132.8083, -144.4641, -111.6421, -141.9807, -142.7692,
         -158.6566, -140.4870, -146.3241, -144.0021]], device='cuda:0')
tensor([[-101.9027, -123.9682, -128.7325, -103.0683, -119.9434, -124.2638,
         -138.7270, -123.5685, -129.4210, -136.7239]], device='cuda:0')
tensor([[ -89.1811, -108.4543, -104.6276, -117.1202,  -88.8084, -113.4260,
         -120.0657, -119.2597, -111.1928, -114.3940]], device='cuda:0')
tensor([[-107.7883, -129.3340, -127.2630, -115.4828, -117.4208, -128.4313,
         -140.3727, -121.3809, -127.8248, -139.2729]], device='cuda:0')
tensor([[ -76.5300, -109.1930, -110.9520, -107.4778, -101.5686, -103.1810,
         -114.9386, -111.6931, -113.6769, -

In [173]:
def get_ee_classification_input(classname, filename, model):
    image = Image.open(f"..\\torch_normal_adv_samples\\results_transfer\\inputs\\{classname}\\{filename}")
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        predclass = torch.argmax(output, dim=1).item()
    return predclass

def get_ee_classification_set_input(directory_path, model):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        expected = update_class_indices[class_indices[classname]]
        cnt = 0
        correct = 0
        # print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            eec = get_ee_classification(classname, filename, model)
            if expected == eec:
                correct += 1
            cnt += 1
        print(f"Class: {classname} has got accuracy of {correct / cnt * 100}%")

In [175]:
get_ee_classification_set_input("..\\torch_normal_adv_samples\\results_transfer\\inputs", model075)

tensor([[-52.8601, -79.5434, -75.9896, -81.3551, -79.2835, -78.2073, -75.3502,
         -78.9040, -79.2891, -81.9402]], device='cuda:0')
tensor([[-47.5320, -78.1771, -77.2189, -77.4546, -73.6466, -75.9219, -80.2771,
         -79.8785, -82.5942, -79.6048]], device='cuda:0')
tensor([[ -66.3998,  -94.2035,  -98.3464, -103.6556,  -96.9234,  -98.6452,
          -95.0345,  -91.6456,  -98.5464,  -94.7164]], device='cuda:0')
tensor([[-55.4665, -83.1903, -83.6980, -84.2922, -89.3515, -89.8405, -86.0942,
         -82.4067, -85.4348, -87.0331]], device='cuda:0')
tensor([[-64.3379, -94.8342, -92.4719, -99.0503, -96.9462, -92.4189, -94.6388,
         -94.0480, -97.0052, -95.9406]], device='cuda:0')
tensor([[ -75.4316, -109.7291, -110.4411, -103.2524, -110.9005, -109.3376,
         -109.9235, -106.5174, -104.5240, -116.0683]], device='cuda:0')
Class: n01440764 has got accuracy of 80.0%
tensor([[ -95.8262,  -65.1802,  -96.2365,  -96.8477, -100.9924,  -97.0075,
          -96.6581,  -97.1327,  -98.3779

In [46]:
temp = Image.open("..\\torch_normal_adv_samples\\results_transfer\\outputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)

In [41]:
x = list(model025.model.children())[:4]

In [49]:
output = temp
for layer in x:
    layer.to(device)
    output = layer(output)

output = torch.flatten(output, start_dim=1)

In [50]:
output.shape

torch.Size([1, 200704])

In [55]:
y = list(list(model025.model.children())[4].children())[1:2]

In [56]:
y

[Linear(in_features=200704, out_features=1, bias=True)]

In [58]:
output = y[0](output)

In [59]:
output

tensor([[-0.2870]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [97]:
layers = x + [nn.Flatten()] + y
smallmodel = nn.Sequential(*layers)

In [98]:
smallmodel

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): Linear(in_features=200704, out_features=1, bias=True)
)

In [99]:
smallmodel(temp)

tensor([[-0.2870]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [531]:
def generate_adversaries(model, baseImage, delta, steps=50, epsilon=0.005, target=0.75, mode='up'):
    for _ in range(steps):
        # watch gradients
        baseImage.requires_grad = True
        delta.requires_grad = True
        
        # add perturbation
        adversary = baseImage + delta
        
        # run perturbed image through the model and calculate the loss
        predictions = model(adversary)
        print(predictions)
        
        # calculate regularization term to move output towards target_value
        if mode == "up":
            deviation = predictions - target
        else:
            deviation = target - predictions
        # print(deviation)
        sign = torch.sign(deviation)
        loss_regularization = torch.abs(deviation) * sign
        
        loss = LAMDBA_ * loss_regularization.mean()
        
        # calculate gradients of loss with respect to perturbation
        model.zero_grad()
        loss.backward()
        
        # update the perturbation vector
        delta_grad = delta.grad.detach().sign()  # take sign of the gradient
        delta.data = (delta + epsilon * delta_grad).clamp(-epsilon, epsilon)  # update delta and clip
        
        # clamp the perturbation vector
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
    return delta

In [153]:
image = Image.open("..\\torch_normal_adv_samples\\results_transfer\\inputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
image = transform(image)
image = image.unsqueeze(0).to(device)

In [154]:
image.shape

torch.Size([1, 3, 224, 224])

In [155]:
image.requires_grad = True
delta = torch.zeros_like(image, requires_grad=True).to(device)

adversary = generate_adversaries(smallmodel, image, delta, steps=20, epsilon=0.005)

tensor([[-0.3099]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.4855]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.5618]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6011]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6230]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6359]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6425]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6470]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6496]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6522]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6536]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6550]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6560]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6570]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6575]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6583]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(

In [156]:
adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_image = (adversary_image * 255).astype('uint8')
adversary_image = Image.fromarray(adversary_image)

In [157]:
smallmodel.to(device)
new = transform(adversary_image).unsqueeze(0).to(device)
output = smallmodel(new)
print(output)

tensor([[0.8100]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [158]:
with torch.no_grad():
    output = model025(new)
    predclass = torch.argmax(output, dim=1).item()

In [159]:
predclass

3

In [160]:
adversary_image.save('output.JPEG')

In [161]:
adversary_pil = adversary.squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))

In [162]:
adversary_pil.save("change.JPEG")

In [163]:
model025.num_exits_per_module

[1, 0, 0, 0]

In [530]:
def gen_sample(image_path, mode):
    image = Image.open(image_path)
    # print(image_path)
    image = transform(image).unsqueeze(0).to(device)

    # some images are not transformable normally?
    try:
        # save before img
        newimg = Image.open(image_path)
        newimg = transform(newimg)
        
    except TypeError:
        return None

    # create perturbation
    image.requires_grad = True
    delta = torch.zeros_like(image, requires_grad=True).to(device)

    adversary = generate_adversaries(smallmodel, image, delta, steps=20, epsilon=0.005, mode=mode)  

    # create adv image
    adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
    adversary_image = (adversary_image * 255).astype('uint8')
    adversary_image = Image.fromarray(adversary_image)
    

    new = transform(adversary_image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model025(new)
        predclass = torch.argmax(output, dim=1).item()
        return predclass

In [170]:
def gen_sample_set(directory_path):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        print(classname)
        cnt = 0
        correct = 0
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            # filename = file.split("\\")[-1]
            expected = update_class_indices[class_indices[classname]]
            eec = gen_sample(file, "up")
            if expected == eec:
                correct += 1
            cnt += 1
        print(f"Class: {classname} has got accuracy of {correct / cnt * 100}%")

In [171]:
gen_sample_set("..\\torch_normal_adv_samples\\results_transfer\\inputs")

n01440764
Class: n01440764 has got accuracy of 34.0%
n02102040
Class: n02102040 has got accuracy of 26.0%
n02979186
Class: n02979186 has got accuracy of 20.0%
n03000684
tensor([[-104.6533, -105.0534, -105.3958,  -77.0286, -106.5709, -100.7227,
         -108.4825, -107.6332, -107.9240, -104.1230]], device='cuda:0')
Class: n03000684 has got accuracy of 55.00000000000001%
n03028079
Class: n03028079 has got accuracy of 40.0%
n03394916
Class: n03394916 has got accuracy of 75.0%
n03417042
Class: n03417042 has got accuracy of 25.0%
n03425413
Class: n03425413 has got accuracy of 57.99999999999999%
n03445777
Class: n03445777 has got accuracy of 19.0%
n03888257
Class: n03888257 has got accuracy of 5.0%


In [482]:
directory_path = "..\\torch_normal_adv_samples\\results_transfer\\inputs"

In [515]:
temp = Image.open("..\\torch_normal_adv_samples\\results_transfer\\inputs\\n01440764\\ILSVRC2012_val_00000293.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)
# with torch.no_grad():
#     tempout = model075(temp)
#     temppredclass = torch.argmax(tempout, dim=1).item()
# print(temppredclass)

In [500]:
model075.num_exits_per_module

[0, 1, 0, 0]

In [516]:
x = list(model075.model.children())[:5]

In [517]:
y = list(list(model075.model.children())[5].children())[1:2]

In [518]:
output = temp
for layer in x:
    layer.to(device)
    output = layer(output)

output = torch.flatten(output, start_dim=1)

In [519]:
output = y[0](output)

In [520]:
output

tensor([[0.2671]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [521]:
layers = x + [nn.Flatten()] + y
smallmodel2 = nn.Sequential(*layers)

In [522]:
smallmodel2

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): OptionalExitModule(
    (module): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

In [663]:
temp = Image.open("..\\torch_normal_adv_samples\\results_transfer\\inputs\\n01440764\\ILSVRC2012_val_00000293.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)

In [664]:
temp.requires_grad = True
delta = torch.zeros_like(temp, requires_grad=True).to(device)

In [665]:
adversary = generate_adversaries(smallmodel2, temp, delta, steps=2, epsilon=0.01, target=-10.0, mode="down")

tensor([[0.2671]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-2.9448]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [669]:
adversary_image = (temp + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_image = (adversary_image * 255).astype('uint8')
adversary_image = Image.fromarray(adversary_image)

In [670]:
adversary_image.save('push1.JPEG')

In [671]:
adversary_pil = adversary.squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))
adversary_pil.save("push1change.JPEG")

In [667]:
smallmodel2.to(device)
new = transform(adversary_image).unsqueeze(0).to(device)
output = smallmodel2(new)
print(output)

tensor([[-0.2051]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [672]:
with torch.no_grad():
    output = model075(new)
    pred = torch.argmax(output, dim=1).item()

In [673]:
model075.num_exits_per_module

[0, 0, 1, 0]

In [586]:
list(orgmodel.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [617]:
newx = list(model075.model.children())[:6]

In [618]:
newy = list(list(model075.model.children())[6].children())[1:2]

In [674]:
output = new
for layer in newx:
    layer.to(device)
    output = layer(output)

output = torch.flatten(output, start_dim=1)

In [675]:
output = newy[0](output)

In [676]:
output

tensor([[0.1588]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [677]:
newlayers = newx + [nn.Flatten()] + newy
smallmodel3 = nn.Sequential(*newlayers)

In [603]:
smallmodel3

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): OptionalExitModule(
    (module): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

In [678]:
new.requires_grad = True
delta = torch.zeros_like(new, requires_grad=True).to(device)

In [679]:
adversary = generate_adversaries(smallmodel3, new, delta, steps=2, epsilon=0.01, target=-10.0, mode="down")

tensor([[0.1588]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-2.2508]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [685]:
adversary_image = (new + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_image = (adversary_image * 255).astype('uint8')
adversary_image = Image.fromarray(adversary_image)

In [686]:
adversary_image.save('push2.JPEG')

In [687]:
adversary_pil = adversary.squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))
adversary_pil.save("push2change.JPEG")

In [684]:
smallmodel3.to(device)
new2 = transform(adversary_image).unsqueeze(0).to(device)
output = smallmodel3(new2)
print(output)

tensor([[-2.3252]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [658]:
def generate_adversaries_down(model, baseImage, delta, steps=50, epsilon=0.005, target=0.75, mode='up'):
    for _ in range(steps):
        # watch gradients
        baseImage = baseImage.clone().detach().requires_grad_(True)
        delta = delta.clone().detach().requires_grad_(True)
        
        # add perturbation
        adversary = baseImage + delta
        
        # run perturbed image through the model and calculate the loss
        predictions = model(adversary)
        # print(predictions)
        
        # calculate regularization term to move output towards target_value
        if mode == "up":
            deviation = predictions - target
        else:
            deviation = target - predictions
        # print(deviation)
        sign = torch.sign(deviation)
        loss_regularization = torch.abs(deviation) * sign
        
        loss = LAMDBA_ * loss_regularization.mean()
        
        # calculate gradients of loss with respect to perturbation
        model.zero_grad()
        loss.backward()
        
        # update the perturbation vector
        delta_grad = delta.grad.detach().sign()  # take sign of the gradient
        delta.data = (delta + epsilon * delta_grad).clamp(-epsilon, epsilon)  # update delta and clip
        
        # clamp the perturbation vector
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
    return delta

In [640]:
def gen_sample_down(image_path, mode):
    image = Image.open(image_path)
    # print(image_path)
    image = transform(image).unsqueeze(0).to(device)

    # some images are not transformable normally?
    try:
        # save before img
        newimg = Image.open(image_path)
        newimg = transform(newimg)
        
    except TypeError:
        return None

    # create perturbation
    # image.requires_grad = True
    delta = torch.zeros_like(image, requires_grad=True).to(device)

    adversary = generate_adversaries_down(smallmodel2, image, delta, steps=2, epsilon=0.01, mode=mode)

    # create adv image
    image = (image + adversary)
    # image.requires_grad = True
    delta = torch.zeros_like(image, requires_grad=True).to(device)

    adversary = generate_adversaries_down(smallmodel3, image, delta, steps=2, epsilon=0.01, mode=mode)  

    # create adv image
    adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
    adversary_image = (adversary_image * 255).astype('uint8')
    adversary_image = Image.fromarray(adversary_image)
    

    new = transform(adversary_image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model075(new)
        predclass = torch.argmax(output, dim=1).item()
        return predclass, model075.num_exits_per_module

In [641]:
gen_sample_down("..\\torch_normal_adv_samples\\results_transfer\\inputs\\n01440764\\ILSVRC2012_val_00000293.JPEG", "down")

tensor([[0.2671]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-2.9448]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.3160]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-2.1837]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[ -61.3774,  -94.7439,  -92.0569, -101.3762,  -85.1474,  -95.4497,
         -105.2471, -101.1099, -102.9531,  -97.7021]], device='cuda:0')


(0, [0, 0, 0, 1])

In [661]:
def gen_sample_set_down(directory_path):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        print(classname)
        cnt = 0
        correct = 0
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            temp = Image.open(file)
            temp = transform(temp)
            temp = temp.unsqueeze(0).to(device)
            with torch.no_grad():
                model075(temp)
            if model075.num_exits_per_module == [0, 1, 0, 0]:
                try:
                    print(gen_sample_down(file, "down"))
                    correct += 1
                except Exception:
                    continue
                finally:
                    cnt += 1
        print(f"Class: {classname} found {correct / cnt * 100}% possible pushbacks")
        #     if expected == eec:
        #         correct += 1
        #     cnt += 1
        # print(f"Class: {classname} has got accuracy of {correct / cnt * 100}%")

In [662]:
gen_sample_set_down("..\\torch_normal_adv_samples\\results_transfer\\inputs")

n01440764
tensor([[ -61.3774,  -94.7439,  -92.0569, -101.3762,  -85.1474,  -95.4497,
         -105.2471, -101.1099, -102.9531,  -97.7021]], device='cuda:0')
(0, [0, 0, 0, 1])
tensor([[ -72.9966, -101.4634,  -96.7159,  -97.9323, -102.7364, -101.7842,
          -98.8019, -102.3126, -106.1333, -104.1015]], device='cuda:0')
(0, [0, 0, 0, 1])
tensor([[ -99.8027, -131.3565, -132.0831, -119.1755, -135.8907, -137.2115,
         -143.0777, -129.1062, -131.8780, -133.2953]], device='cuda:0')
(0, [0, 0, 0, 1])
tensor([[ -65.3208, -102.4579, -102.3683,  -96.2335, -104.9398, -102.2456,
         -103.4231, -101.4574, -103.4104, -107.3714]], device='cuda:0')
(0, [0, 0, 0, 1])
tensor([[ -93.5174, -109.5305, -116.8442, -113.8937, -115.0268, -115.7821,
         -124.4132, -118.4648, -125.7894, -121.4858]], device='cuda:0')
(0, [0, 0, 0, 1])
tensor([[ -65.8906,  -94.7626,  -94.8087,  -91.1728,  -93.1509,  -95.6574,
          -98.8203,  -94.6538, -101.0311,  -98.7437]], device='cuda:0')
(0, [0, 0, 0, 1])
